<a href="https://colab.research.google.com/github/aaaven/stylegan2encoder/blob/master/Face2Animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool02 - Create the Animation 
with "First Order Motion Model for Image Animation"

## generate Animated video

**Clone repository**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 249, done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 249
Receiving objects: 100% (249/249), 72.12 MiB | 21.23 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive



**Load driving video and source image**

In [5]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

prefix = '/content/gdrive/MyDrive/0-Colab-Assets/face2animation/'#@param {type:"string"}
source_image_filename = 'qianlong.png'#@param {type:"string"}
driving_video_filename = 'driving_0.mp4'#@param {type:"string"}
source_image = imageio.imread(prefix + source_image_filename)
reader = imageio.get_reader(prefix + driving_video_filename)


#Resize image and video to 256x256
video_size = "256" #@param [128, 256, 512, 720, 1024]
v_size = int(video_size)
source_image = resize(source_image, (v_size, v_size))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (v_size, v_size))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=33, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Perform image animation: Driving Video 'drives' the Source Image**



In [6]:
# Create a model and load checkpoints
from demo import load_checkpoints
checkpoint_path = '/vox-cpk.pth.tar' #@param ["/vox-cpk.pth.tar", "/vox-adv-cpk.pth.tar"]
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=prefix+checkpoint_path)

# run model and inference
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
generated_video_name = 'animated-256-qianlong.mp4'#@param {type:"string"}
if generated_video_name == '':
  generated_video_name = 'animated-'+source_image_filename.split('.',1)[0]+'.mp4' 
generated_video_path = prefix+generated_video_name

imageio.mimsave(generated_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)
print('video:' +generated_video_name + ' has been saved at ' + prefix)

print('or you can also download the three in one video by clicking the video below')
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 210/210 [00:04<00:00, 44.52it/s]


video:animated-256-qianlong.mp4 has been saved at /content/gdrive/MyDrive/0-Colab-Assets/mona_lisa_eyes/
or you can also download the three in one video by clicking the video below


In [ ]:
# use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video.
# predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
# HTML(display(source_image, driving_video, predictions).to_html5_video())

 examples: <!--https://drive.google.com/drive/*folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing or  https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing* -->